# Upcapacited Facility Location Problem

## Background

* You are deciding on five new potiental sites to build an FC.
* You have a set of customer demands (individual customer orders) that you plan to fulfill from these FCs
* Every customer order incurs a shipping cost. 
* Shipping cost for each cutomer order = (distance from FC to customer destination)/$100
* We assume all customer orders are the same (no product variation)
* Each facility that is opened will incur a one time fixed cost for construction
* We assume facilities are uncapacitited

## Problem statement
* Which five sites should you open to minimize overall costs while still being able to statify customer demand?


# Mathematical Formulation

### <font color=green>  Decision Variables

<div class="alert alert-block alert-warning">
<b>Decision Variables:</b> $x_{ij}$ and $y_{c}$

Let $x_{ij}$ = the number of orders to ship from facility ${i}$ to customer destination ${j}$<br>
Let $y_{i}$ = 1 if facility ${i}$ is selected, 0 otherwise<br>
    
<br>
$x_{ij}\in {\rm I\!R}$<br>
$y_{i}\in (0,1)$<br>
</div>

### <font color=green>  Parameters

<div class="alert alert-block alert-warning">
Let $I$ be the set of possible FC locations to choose from <br>
Let $J$ be the set of all customers <br>
Let $N$ = the total number of facilities to open<br><br>

Let $f_i$ be the setup cost for constructing facility $i$<br>
Let $c_{ij}$ be the cost of shipping an order from facility $i$ to customer $j$<br>
Let $d_{j}$ be the demand from customer j<br>

</div>

### <font color=green>  Objective Function

<div class="alert alert-block alert-warning">
<b>Objective Function:</b> Minimize Z = startup_cost + transportation_cost


$$ Z = \sum_{j \: \in \:I} f_iy_i + \sum_{i \: \in \:I}\sum_{j \: \in \:J}c_{ij}x_{ij}$$ 

</div>

### <font color=green>  Constraints

<div class="alert alert-block alert-warning">
<b>Cosntratint 1:</b> all demand must be met

$$ 
\sum_{i \: \in \:I} x_{ij} = d_j 
\qquad \forall  \enspace j \: \in \: J
$$ 

</div>

<div class="alert alert-block alert-warning">
<b>Cosntratint 2:</b> shipments can only be made from sites that have been selected to open

$$ 
\sum_{j \: \in \:J} x_{ij} \leq M_iy_i
\qquad \forall  \enspace i \: \in \: I; 
$$ 

</div>

<div class="alert alert-block alert-warning">
<b>Cosntratint 3:</b> Exactly N new sites must be opened (5)

$$ 
\sum_{i \: \in \:I} y_i = N 
$$ 


</div>

<div class="alert alert-block alert-warning">
<b>Cosntratint 4:</b>

$$ 
x_{ij} \geq 0 
\qquad \forall  \enspace i \: \epsilon \: I; 
\enspace j \: \epsilon \: J
$$ 

$$ 
y_{j} \in {0,1}
$$ 


</div>

# CPlex/Python LP Model

### Import packages

In [ ]:
import pandas as pd
import numpy as np
from docplex.mp.model import Model
import math

### Intializing the data

In [ ]:
# reading in external data
facility_df = pd.read_csv('facility_data.csv')
customer_df = pd.read_csv('customer_data.csv')

In [ ]:
# caluclating distance
facility_df['key'] = 1
customer_df['key'] = 1
result = pd.merge(facility_df, customer_df, on ='key').drop("key", 1) 
result['temp_x'] = (result['facility_x'] - result['customer_x'])**2 
result['temp_y'] = (result['facility_y'] - result['customer_y'])**2 
result['temp_x_y'] = (result['temp_x'] + result['temp_y'])
result['distance'] = round(np.sqrt(result['temp_x_y']),2)
distance_df = result.copy()[['facility','customer','distance']]
distance_df.head()

In [ ]:
# for use in model 
distance = dict([((t.facility, t.customer),t.distance ) for t in distance_df.itertuples()])
setup_cost = dict([((t.facility),t.setup_cost ) for t in facility_df.itertuples()])
demand = dict([((t.customer),t.demand ) for t in customer_df.itertuples()])
fc =  set(facility_df['facility'])
customer =  set(customer_df['customer'])
edges = [(i, j) for i in fc for j in customer]
N = 5 # number of sits to open
M = round(distance_df['distance'].max()+100,0)

### Create the model

In [ ]:
m=Model('TSP')

#### <font color=green>  Decision Variables

In [ ]:
x = m.continuous_var_dict(edges, name ='assignment')  
y = m.binary_var_dict(fc, name = 'fc')

#### <font color=green>  Objective Function

In [ ]:
trans_cost = m.sum(distance[e]*x[e] for e in edges)
startup_cost = m.sum(setup_cost[i]*y[i] for i in fc)
m.minimize(startup_cost + trans_cost)

#### <font color=green>  Constraints

In [ ]:
# Constraint 1: all demand must be met
for j in customer:
    m.add_constraint(m.sum(x[(i,j)] for i in fc) == demand[j], ctname='demand_%d'%j)

# Constraint 2: shipments can only be made if the site has been opened
for i in fc:
    m.add_constraint(m.sum(x[(i,j)] for j in customer) <= M*y[i], ctname='lane_%d'%i)
    
# Constraint 3: exactly five sites must be opened
m.add_constraint(m.sum(y[(i)] for i in fc) == N, ctname='num_sites')
    
       

In [ ]:
print(m.export_to_string())

In [ ]:
m.parameters.timelimit=120
m.parameters.mip.strategy.branch=1
m.parameters.mip.tolerances.mipgap=0.15

solution = m.solve(log_output=True)

In [ ]:
m.get_solve_status()


In [ ]:
solution.display()

In [ ]:
# Export results to csv
import os

base_dir = os.getcwd()

def export_soln_to_csv(df, model_name = 'untitled'):
    """ model refers to model object from docplex.mp.model"""

    try:
        os.mkdir(os.path.join(base_dir, 'output'))
    except:
        pass

    filename = 'output/' + 'soln_' + model_name + '.csv'
    solution_output = os.path.join(os.getcwd(), filename)
    df.to_csv(solution_output, index=False)